In [1]:
import librosa
import torch # 2.7.0 han, bharat is 2.5.0
import coremltools as ct # 8.3 on bharat , 7.2 Han
# has to be Python 3.12 , bharat python 3.10.12 . can't be python 3.13
import numpy as np
import torch.nn as nn
from coremltools.converters.mil import input_types
print(ct.__version__)
print(torch.__version__)


8.3.0
2.5.0


In [2]:
# model_name = "nvidia/parakeet-tdt-0.6b-v2"
model_name = "nvidia/parakeet-tdt-0.6b-v3"
audio, sr = librosa.load("first_10_seconds.wav", sr=16000)

audio_length = torch.tensor(audio.shape)
audio_tensor = torch.from_numpy(audio).unsqueeze(0)

### Load ASR module and transcribe

In [3]:
import nemo.collections.asr as nemo_asr
print(nemo_asr.__version__)

asr_model = nemo_asr.models.ASRModel.from_pretrained(model_name=model_name)

2.3.1
[NeMo I 2025-08-20 17:07:43 mixins:181] Tokenizer SentencePieceTokenizer initialized with 8192 tokens


[NeMo W 2025-08-20 17:07:46 modelPT:180] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_lhotse: true
    skip_missing_manifest_entries: true
    input_cfg: null
    tarred_audio_filepaths: null
    manifest_filepath: null
    sample_rate: 16000
    shuffle: true
    num_workers: 2
    pin_memory: true
    max_duration: 10.0
    min_duration: 1.0
    text_field: answer
    batch_duration: null
    max_tps: null
    use_bucketing: true
    bucket_duration_bins: null
    bucket_batch_size: null
    num_buckets: 30
    bucket_buffer_size: 20000
    shuffle_buffer_size: 10000
    
[NeMo W 2025-08-20 17:07:46 modelPT:187] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation 

[NeMo I 2025-08-20 17:07:46 features:305] PADDING: 0
[NeMo I 2025-08-20 17:07:49 rnnt_models:226] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}
[NeMo I 2025-08-20 17:07:49 rnnt_models:226] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}


[NeMo W 2025-08-20 17:07:49 tdt_loop_labels_computer:300] No conditional node support for Cuda.
    Cuda graphs with while loops are disabled, decoding speed will be slower
    Reason: CUDA is not available


[NeMo I 2025-08-20 17:07:49 rnnt_models:226] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}


[NeMo W 2025-08-20 17:07:49 tdt_loop_labels_computer:300] No conditional node support for Cuda.
    Cuda graphs with while loops are disabled, decoding speed will be slower
    Reason: CUDA is not available


[NeMo I 2025-08-20 17:07:52 save_restore_connector:275] Model EncDecRNNTBPEModel was successfully restored from /Users/kikow/.cache/huggingface/hub/models--nvidia--parakeet-tdt-0.6b-v3/snapshots/bc3e42c344d9127e85c2d2f92be914f57d741b59/parakeet-tdt-0.6b-v3.nemo.


In [4]:
asr_model.cfg["decoder"]

{'_target_': 'nemo.collections.asr.modules.RNNTDecoder', 'normalization_mode': None, 'random_state_sampling': False, 'blank_as_pad': True, 'prednet': {'pred_hidden': 640, 'pred_rnn_layers': 2, 't_max': None, 'dropout': 0.2}, 'vocab_size': 8192}

In [5]:
output = asr_model.transcribe(['first_10_seconds.wav'])
output

Transcribing: 100%|██████████| 1/1 [00:00<00:00,  1.66it/s]


[Hypothesis(score=0.0, y_sequence=tensor([], dtype=torch.int64), text='', dec_out=None, dec_state=None, timestamp=[], alignments=None, frame_confidence=None, token_confidence=None, word_confidence=None, length=0, y=None, lm_state=None, lm_scores=None, ngram_lm_state=None, tokens=None, last_token=None, token_duration=None, last_frame=None)]

In [6]:
asr_model.eval()

EncDecRNNTBPEModel(
  (preprocessor): AudioToMelSpectrogramPreprocessor(
    (featurizer): FilterbankFeatures()
  )
  (encoder): ConformerEncoder(
    (pre_encode): ConvSubsampling(
      (out): Linear(in_features=4096, out_features=1024, bias=True)
      (conv): Sequential(
        (0): Conv2d(1, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=256)
        (3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
        (4): ReLU(inplace=True)
        (5): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=256)
        (6): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
        (7): ReLU(inplace=True)
      )
    )
    (pos_enc): RelPositionalEncoding(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): ModuleList(
      (0-23): 24 x ConformerLayer(
        (norm_feed_forward1): LayerNorm((1024,), eps=1e-05, element

### Audio to MelSpectrogram

In [10]:
class AudioToMelSpectrogramPreprocessor(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.sample_rate=16000
        self.preemph = 0.97
        self.guard = 0
        self.mag_power = 2.0
        self.log_zero_guard_value = 5.960464477539063e-08
        self.n_fft = 512
        self.hop_length = 160
        self.win_length = 400
        self.center = True
        self.nfilt = 128
        self.lowfreq = 0
        self.highfreq = self.sample_rate / 2
        self.mel_norm="slaney"
        self.pad_value = 0.0
        self.CONSTANT = 1e-05

        self.register_buffer(
            "filterbanks",
            torch.tensor(
                librosa.filters.mel(
                    sr=self.sample_rate, n_fft=self.n_fft, n_mels=self.nfilt, fmin=self.lowfreq, fmax=self.highfreq, norm=self.mel_norm
                ),
                dtype=torch.float,
            ).unsqueeze(0)
        )
        self.register_buffer("window", torch.hann_window(self.win_length, periodic=False))

    def stft(self, x):
        return torch.stft(
            x,
            n_fft=self.n_fft,
            hop_length=self.hop_length,
            win_length=self.win_length,
            center=True,
            window=self.window,
            return_complex=True,
        )

    def get_seq_len(self, seq_len):
        # Assuming that center is True is stft_pad_amount = 0
        pad_amount = self.n_fft // 2 * 2
        seq_len = torch.floor_divide((seq_len + pad_amount - self.n_fft), self.hop_length) + 1
        return seq_len.to(dtype=torch.long)

    def normalize_batch(self, x, seq_len):
        x_mean = None
        x_std = None

        batch_size = x.shape[0]
        max_time = x.shape[2]

        time_steps = torch.arange(max_time, device=x.device).unsqueeze(0).expand(batch_size, max_time)
        # time_steps = torch.arange(x.size(2), device=x.device).unsqueeze(0).repeat(x.size(0), 1)

        valid_mask = time_steps < seq_len.unsqueeze(1)
        x_mean_numerator = torch.where(valid_mask.unsqueeze(1), x, 0.0).sum(axis=2)
        x_mean_denominator = valid_mask.sum(axis=1)
        x_mean = x_mean_numerator / x_mean_denominator.unsqueeze(1)

        # Subtract 1 in the denominator to correct for the bias.
        x_std = torch.sqrt(
            torch.sum(torch.where(valid_mask.unsqueeze(1), x - x_mean.unsqueeze(2), 0.0) ** 2, axis=2)
            / (x_mean_denominator.unsqueeze(1) - 1.0)
        )

        # make sure x_std is not zero
        x_std += self.CONSTANT
        return (x - x_mean.unsqueeze(2)) / x_std.unsqueeze(2), x_mean, x_std

    def audio_preprocessor_melspectrogram(self, input_signal, input_length):
        seq_len = self.get_seq_len(input_length)

        x = input_signal
        x = torch.cat((x[:, 0].unsqueeze(1), x[:, 1:] - self.preemph * x[:, :-1]), dim=1)  # [1, 160000]
        x = self.stft(x)    # [1, 257, 1001]
        x = torch.view_as_real(x)
        x = torch.sqrt(x.pow(2).sum(-1) + self.guard)
        x = x.pow(self.mag_power)
        x = torch.matmul(self.filterbanks, x)  # [1, 128, 1001]
        x = torch.log(x + self.log_zero_guard_value)
        x, _, _ = self.normalize_batch(x, seq_len)

        max_len = x.size(-1)
        mask = torch.arange(max_len, device=x.device)
        mask = mask.repeat(x.size(0), 1) >= seq_len.unsqueeze(1)
        x = x.masked_fill(mask.unsqueeze(1).type(torch.bool).to(device=x.device), self.pad_value)  # torch.Size([1, 128, 1001])
        return (x, seq_len)

    def forward(self, input_signals, input_length):
        # This uses only the joint network (encoder+decoder → logits)
        return self.audio_preprocessor_melspectrogram(input_signals, input_length)

audio_melspectrogram_model = AudioToMelSpectrogramPreprocessor()

example_inputs = (
    audio_tensor,
    audio_length
)

traced_melspectrogram = torch.jit.trace(audio_melspectrogram_model, example_inputs)

In [11]:
import coremltools as ct
import numpy as np


# Convert to CoreML model
traced_melspectrogram.eval()
mlmodel_melspectrogram = ct.convert(
    traced_melspectrogram,
    inputs=[
        ct.TensorType(name="audio_signal", shape=(1, ct.RangeDim(1, 160000)), dtype=np.float64),
        ct.TensorType(name="audio_length", shape=(1,), dtype=np.int32),
    ],
    outputs=[
        ct.TensorType(name="melspectrogram"),
        ct.TensorType(name="melspectrogram_length"),
    ],
    convert_to="mlprogram",  # more flexible backend
    minimum_deployment_target=ct.target.iOS15  # iOS13 doesn't support mlprogram
)


Tuple detected at graph output. This will be flattened in the converted model.
fp64 dtype input audio_signal down casted to fp32.
Running MIL default pipeline:   0%|          | 0/89 [00:00<?, ? passes/s]Saving value type of float64 into a builtin type of fp32, might lose precision!
Saving value type of float64 into a builtin type of fp32, might lose precision!
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 822.09 passes/s]


In [15]:
mlmodel_melspectrogram.save("Melspectrogram_v2.mlpackage")

In [16]:
mlmodel_spectrogram = ct.models.MLModel("Melspectrogram_v2.mlpackage", compute_units=ct.ComputeUnit.CPU_AND_GPU)

# Inputs must be a dictionary with keys matching model input names
melspectrogram_dict = {
    "audio_signal": audio_tensor.numpy().astype(np.float64),
    "audio_length": audio_length.numpy().astype(np.int32),
}

output_spectrogram = mlmodel_spectrogram.predict(melspectrogram_dict)
output_spectrogram

{'melspectrogram_length': array([1001], dtype=int32),
 'melspectrogram': array([[[ 7.6207867 ,  2.4789796 ,  0.21312211, ...,  1.3899976 ,
           4.3642735 ,  8.626841  ],
         [ 8.278219  ,  2.3035228 ,  0.677722  , ...,  0.3853632 ,
           4.041254  ,  9.412934  ],
         [ 6.487728  ,  1.7332808 ,  0.87147015, ..., -1.091425  ,
           2.5147805 ,  7.362334  ],
         ...,
         [-1.0946904 , -0.85021734,  1.1737236 , ...,  0.7718007 ,
          -0.53026676, -2.3964825 ],
         [-0.61513346,  0.6595316 ,  1.1478447 , ...,  0.54370254,
          -2.6312695 , -1.3411158 ],
         [-2.459402  , -0.8596429 ,  0.8162584 , ...,  1.056027  ,
          -1.7531265 , -1.0641257 ]]], dtype=float32)}

In [17]:
abc = traced_melspectrogram(audio_tensor, audio_length)
abc[0]

tensor([[[ 7.7770,  2.6692,  0.3476,  ...,  1.4008,  4.4737,  8.7912],
         [ 8.3773,  2.3284,  0.5868,  ...,  0.4289,  4.1233,  9.5138],
         [ 6.4658,  1.5833,  0.7371,  ..., -0.7741,  2.5679,  7.3296],
         ...,
         [-1.0955, -0.8484,  1.1707,  ...,  0.7711, -0.5327, -2.4015],
         [-0.6102,  0.6677,  1.1487,  ...,  0.5485, -2.6254, -1.3346],
         [-2.4561, -0.8580,  0.8206,  ...,  1.0533, -1.7486, -1.0644]]])

### Encoder Conversion to CoreML

In [26]:
class EncoderWrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, audio_signal, length):
        # Get encoder output (shape: [B, H, T])
        output = self.model(audio_signal=audio_signal, length=length)
        if isinstance(output, tuple):
            encoder_output, encoder_length = output
        else:
            encoder_output = output
            encoder_length = length  # Use input length if not provided

        # Transpose to [B, T, H] for compatibility with decoder
        # CoreML/Swift expects this format
        encoder_output = encoder_output.transpose(1, 2).contiguous()
        return encoder_output, encoder_length

asr_model.encoder.eval()
model_encoder = EncoderWrapper(asr_model.encoder)

example_inputs = (
    torch.from_numpy(output_spectrogram["melspectrogram"]),
    torch.from_numpy(output_spectrogram["melspectrogram_length"])
)

traced_encoder = torch.jit.trace(model_encoder, example_inputs)

In [ ]:
# Convert to CoreML with dynamic input
traced_encoder.eval()
mlmodel_encoder = ct.convert(
  traced_encoder,
    inputs=[
        ct.TensorType(name="audio_signal", shape=(1, 128, 1001), dtype=np.float32),
        ct.TensorType(name="length", shape=(1,), dtype=int)
    ],
    outputs=[
        ct.TensorType(name="encoder_output"),
        ct.TensorType(name="encoder_output_length"),
    ],
  compute_precision=ct.precision.FLOAT16,
  minimum_deployment_target=ct.target.iOS15
)

# Apply INT8 quantization
op_config = OpLinearQuantizerConfig(
  mode="linear_symmetric",
  granularity="per_channel",
  weight_threshold=512,
)
config = OptimizationConfig(global_config=op_config)

mlmodel_encoder_int8 = linear_quantize_weights(mlmodel_encoder, config=config)

# Save the INT8 model
mlmodel_encoder_int8.save("ParakeetEncoder_v2.mlpackage")
print("✓ Saved INT8 encoder to ParakeetEncoder_v2.mlpackage")
print("  Supports dynamic input: 0.1-10 seconds")

# Test the output shape to verify transpose is working
test_mel = np.random.randn(1, 128, 1001).astype(np.float32)
test_length = np.array([1001], dtype=np.int32)

output = mlmodel_encoder_int8.predict({
  'audio_signal': test_mel,
  'length': test_length
})

print(f"\nVerification:")
print(f"Output shape: {output['encoder_output'].shape}")
print(f"Expected: [1, 126, 1024] (batch, time, hidden)")
if output['encoder_output'].shape == (1, 126, 1024):
  print("✅ Transpose is working correctly!")
else:
  print("❌ Transpose may not be working - check the shape")


Tuple detected at graph output. This will be flattened in the converted model.
Running compression pass linear_quantize_weights: 100%|██████████| 614/614 [00:12<00:00, 48.09 ops/s]
Running MIL frontend_milinternal pipeline: 0 passes [00:00, ? passes/s]
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 24.78 passes/s]


✓ Saved INT8 encoder to ParakeetEncoder_v2.mlpackage
  Supports dynamic input: 0.1-10 seconds

Verification:
Output shape: (1, 126, 1024)
Expected: [1, 126, 1024] (batch, time, hidden)
✅ Transpose is working correctly!


In [ ]:
mlmodel_encoder = ct.models.MLModel("ParakeetEncoder.mlpackage")

# Inputs must be a dictionary with keys matching model input names
input_dict = {
    "melspectrogram": output_spectrogram["melspectrogram"],
    "melspectrogram_length": output_spectrogram["melspectrogram_length"]
}

outputs_encoder = mlmodel_encoder.predict(input_dict)
outputs_encoder

In [ ]:
traced_encoder(torch.from_numpy(output_spectrogram["melspectrogram"]), torch.from_numpy(output_spectrogram["melspectrogram_length"]))

### Decoder conversion to CoreML

In [ ]:
class DecoderWrapper(nn.Module):
    def __init__(self, decoder):
        super().__init__()
        self.embed = decoder.prediction["embed"]
        self.rnn = decoder.prediction["dec_rnn"].lstm  # <== Access underlying LSTM directly

    def forward(self, targets, target_lengths, h_in, c_in):
        embedded = self.embed(targets)  # (B, T, D)
        output, (h_out, c_out) = self.rnn(embedded, (h_in, c_in))  # Run LSTM manually
        return output, h_out, c_out

In [ ]:
wrapped_decoder = DecoderWrapper(asr_model.decoder)

vocab_size = asr_model.decoder.prediction['embed'].num_embeddings
hidden_size = asr_model.cfg["decoder"]["prednet"]["pred_hidden"]     # 640
num_layers = asr_model.cfg["decoder"]["prednet"]["pred_rnn_layers"]  # 2

batch = 1
num_seq = 1

example_inputs = (
    torch.randint(0, vocab_size, (batch, num_seq), dtype=torch.int32),   # targets
    torch.tensor([20], dtype=torch.int32),                    # target_lengths
    torch.zeros(num_layers, num_seq, hidden_size, dtype=torch.float32),                   # h_in
    torch.zeros(num_layers, num_seq, hidden_size, dtype=torch.float32),                   # c_in
)

wrapped_decoder.eval()
traced_decoder = torch.jit.trace(wrapped_decoder, example_inputs)

In [ ]:

traced_decoder.eval()
mlmodel_decoder = ct.convert(
    traced_decoder,
    inputs=[
        input_types.TensorType(
            name="targets",
            shape=(1, input_types.RangeDim(1, 1000)),  # dynamic length
            dtype=int
        ),
        input_types.TensorType(name="target_lengths", shape=(1,), dtype=int),
        input_types.TensorType(name="h_in", shape=(num_layers, num_seq, hidden_size), dtype=np.float32),
        input_types.TensorType(name="c_in", shape=(num_layers, num_seq, hidden_size), dtype=np.float32)
    ],
    outputs=[
        input_types.TensorType(name="decoder_output"),
        input_types.TensorType(name="h_out"),
        input_types.TensorType(name="c_out"),
    ],
    convert_to="mlprogram",
    minimum_deployment_target=ct.target.iOS15
)

In [ ]:
mlmodel_decoder.save("ParakeetDecoder.mlpackage")

In [ ]:
mlmodel_decoder = ct.models.MLModel("ParakeetDecoder.mlpackage")

# Inputs must be a dictionary with keys matching model input names
input_dict = {
    "targets": example_inputs[0],
    "target_lengths": example_inputs[1],
    "h_in": example_inputs[2],
    "c_in": example_inputs[3]
}

output_decoder = mlmodel_decoder.predict(input_dict)
for key in output_decoder.keys():
    print (key, output_decoder[key].shape, output_decoder[key].dtype)

In [ ]:
#!/usr/bin/env python3
"""
SIMPLE COPY-PASTE SCRIPT TO CREATE 30-SECOND ENCODER
Just run: python3 create_30s_encoder.py
"""

import torch
import torch.nn as nn
import numpy as np
import coremltools as ct
from pathlib import Path
import nemo.collections.asr as nemo_asr

print("Creating 30-second encoder...")

# Load the model
nemo_path = Path.home() / ".cache/huggingface/hub/models--nvidia--parakeet-tdt-0.6b-v3/snapshots/bb0964b98adc4b962566f3df9f5f1844011871a3/parakeet-tdt-0.6b-v3.nemo"
asr_model = nemo_asr.models.EncDecRNNTBPEModel.restore_from(str(nemo_path))
asr_model.eval()

# Create 30-second audio sample
sample_audio_30s = torch.randn(1, 480000).float()  # 30 seconds at 16kHz
sample_length_30s = torch.tensor([480000], dtype=torch.long)

# Get 30-second mel-spectrogram
with torch.no_grad():
    mel_30s, mel_length_30s = asr_model.preprocessor(
        input_signal=sample_audio_30s,
        length=sample_length_30s
    )

print(f"Mel-spectrogram shape: {mel_30s.shape}")  # Should be [1, 128, 3001]

# Create encoder wrapper
class EncoderWrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, audio_signal, length):
        return self.model(audio_signal=audio_signal, length=length)

# Wrap and trace encoder
encoder_wrapper = EncoderWrapper(asr_model.encoder)
encoder_wrapper.eval()

traced_encoder = torch.jit.trace(encoder_wrapper, (mel_30s, mel_length_30s))
traced_encoder.eval()

# Convert to CoreML with 30-second shape
print("Converting to CoreML...")
mlmodel_encoder = ct.convert(
    traced_encoder,
    inputs=[
        ct.TensorType(name="melspectrogram", shape=(1, 128, 3001), dtype=np.float32),  # 30 SECONDS!
        ct.TensorType(name="melspectrogram_length", shape=(1,), dtype=np.int32)
    ],
    outputs=[
        ct.TensorType(name="encoder_output"),
        ct.TensorType(name="encoder_output_length"),
    ],
    compute_units=ct.ComputeUnit.CPU_AND_NE,
    minimum_deployment_target=ct.target.iOS15
)

# Save to FluidAudio models directory
output_dir = Path.home() / "Library/Application Support/FluidAudio/Models/parakeet-asr-coreml"
output_dir.mkdir(parents=True, exist_ok=True)

# Backup existing encoder
existing = output_dir / "ParakeetEncoder_transposed_int8.mlpackage"
if existing.exists():
    import shutil
    backup = output_dir / "ParakeetEncoder_10s_backup.mlpackage"
    if not backup.exists():
        shutil.copytree(existing, backup)
        print(f"Backed up existing encoder")

# Save new encoder
output_path = output_dir / "ParakeetEncoder_30s.mlpackage"
mlmodel_encoder.save(str(output_path))
print(f"✅ Saved 30-second encoder to: {output_path}")

# Test it
print("\nTesting encoder...")
model = ct.models.MLModel(str(output_path))

for seconds in [10, 20, 30]:
    frames = seconds * 100 + 1
    test_mel = np.zeros((1, 128, 3001), dtype=np.float32)
    test_mel[:, :, :frames] = np.random.randn(1, 128, frames).astype(np.float32) * 0.1
    test_length = np.array([frames], dtype=np.int32)

    output = model.predict({
        "melspectrogram": test_mel,
        "melspectrogram_length": test_length
    })

    print(f"  {seconds}s input → encoder length: {output['encoder_output_length'][0]}")

print("\n✅ DONE! Your encoder now accepts 30-second inputs!")
print("\nNext: swift build -c release")
print("Test: ./.build/release/fluidaudio asr-benchmark --max-files 5")

### Joint Network

In [ ]:
class JointInferenceWrapper(torch.nn.Module):
    def __init__(self, joint):
        super().__init__()
        self.joint = joint

    def forward(self, encoder_outputs, decoder_outputs):
        # This uses only the joint network (encoder+decoder → logits)
        return self.joint.joint(encoder_outputs, decoder_outputs)

wrapper_joint = JointInferenceWrapper(asr_model.joint)

example_inputs = (
    torch.randn(1, 10, 1024),  # encoder_outputs (B, T, D_enc)
    torch.randn(1, 5, 640),    # decoder_outputs (B, U, D_pred)
)

wrapper_joint.eval()
traced_joint = torch.jit.trace(wrapper_joint, example_inputs)

In [ ]:
# Convert to CoreML model
traced_joint.eval()
mlmodel_joint = ct.convert(
    traced_joint,
    inputs=[
        ct.TensorType(name="encoder_outputs", shape=(ct.RangeDim(1, 100), ct.RangeDim(1, 1025), ct.RangeDim(1, 1024)), dtype=np.float32),
        ct.TensorType(name="decoder_outputs", shape=(ct.RangeDim(1, 100), ct.RangeDim(1, 1025), ct.RangeDim(1, 640)), dtype=np.float32),
    ],
    outputs=[
        ct.TensorType(name="logits"),
    ],
    convert_to="mlprogram",  # more flexible backend
)


In [ ]:
mlmodel_joint.save("RNNTJoint.mlpackage")